In [1]:
data = open('C:/Users/ANKAN/TIMES OF INDIA/First/January.txt',encoding="utf8").read()

In [2]:
texts = []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    texts.append(" ".join(content[1:]))

In [3]:
import pandas
DF = pandas.DataFrame()
DF['text'] = texts

In [4]:
from sklearn.utils import shuffle
DF = shuffle(DF)
DF

,text
73668,"have sent you a verification email. To verify,..."
46036,
70195,Comments ()+^ Back to Top
14328,"arrested for film protest, no show at Hind, Na..."
86195,is scheduled to take place at Ranga Shankara o...
...,...
111791,"| Jan 31, 2019, 00:24 IST"
1251,"After infrastructure, defence, realty and FMCG..."
28296,
13902,Rajesh was the chief guest who inaugurated the...


In [5]:
nan_value = float("NaN")

DF.replace("", nan_value, inplace = True)

DF.dropna(subset = ["text"], inplace = True)

In [6]:
DF


,text
73668,"have sent you a verification email. To verify,..."
70195,Comments ()+^ Back to Top
14328,"arrested for film protest, no show at Hind, Na..."
86195,is scheduled to take place at Ranga Shankara o...
45687,held for sexual abuse of minor
...,...
20201,Seeking resurrection in the politically crucia...
50704,IN WITH
111791,"| Jan 31, 2019, 00:24 IST"
1251,"After infrastructure, defence, realty and FMCG..."


In [7]:
# Load the regular expression library
import re
# Remove punctuation
DF['text_processed'] = DF['text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
DF['text_processed'] = DF['text_processed'].map(lambda x: x.lower())
# Print out the first rows of DF
DF['text_processed'].head()

73668    have sent you a verification email to verify j...
70195                            comments ()+^ back to top
14328    arrested for film protest no show at hind navi...
86195    is scheduled to take place at ranga shankara o...
45687                       held for sexual abuse of minor
Name: text_processed, dtype: object

In [8]:
DF

,text,text_processed
73668,"have sent you a verification email. To verify,...",have sent you a verification email to verify j...
70195,Comments ()+^ Back to Top,comments ()+^ back to top
14328,"arrested for film protest, no show at Hind, Na...",arrested for film protest no show at hind navi...
86195,is scheduled to take place at Ranga Shankara o...,is scheduled to take place at ranga shankara o...
45687,held for sexual abuse of minor,held for sexual abuse of minor
...,...,...
20201,Seeking resurrection in the politically crucia...,seeking resurrection in the politically crucia...
50704,IN WITH,in with
111791,"| Jan 31, 2019, 00:24 IST",| jan 31 2019 00:24 ist
1251,"After infrastructure, defence, realty and FMCG...",after infrastructure defence realty and fmcg t...


In [9]:
# Load the library with the CountVectorizer method
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
# Helper function
def plot_10_most_common_words(count_data, count_vectorizer):
    import matplotlib.pyplot as plt
    words = count_vectorizer.get_feature_names()
    total_counts = np.zeros(len(words))
    for t in count_data:
        total_counts+=t.toarray()[0]
    
    count_dict = (zip(words, total_counts))
    count_dict = sorted(count_dict, key=lambda x:x[1], reverse=True)[0:100]
    words = [w[0] for w in count_dict]
    counts = [w[1] for w in count_dict]
    x_pos = np.arange(len(words)) 
    
    plt.figure(2, figsize=(15, 15/1.6180))
    plt.subplot(title='10 most common words')
    sns.set_context("notebook", font_scale=1.25, rc={"lines.linewidth": 2.5})
    sns.barplot(x_pos, counts, palette='husl')
    plt.xticks(x_pos, words, rotation=90) 
    plt.xlabel('words')
    plt.ylabel('counts')
    plt.show()
# Initialise the count vectorizer with the English stop words
count_vectorizer = CountVectorizer(stop_words='english')
# Fit and transform the processed titles
count_data = count_vectorizer.fit_transform(DF['text_processed'])
# Visualise the 10 most common words
#plot_10_most_common_words(count_data, count_vectorizer)

In [10]:
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
# Load the LDA model from sk-learn
from sklearn.decomposition import LatentDirichletAllocation as LDA
 
# Helper function
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
# Tweak the two parameters below
number_topics = 30
number_words = 15
# Create and fit the LDA model
lda = LDA(n_components=number_topics, n_jobs=-1)
lda.fit(count_data)
# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, number_words)

Topics found via LDA:

Topic #0:
court said government high case order state department issued act report justice filed committee action

Topic #1:
festival event day year kumbh city mela dance pm sankranti said new pongal guests time

Topic #2:
said road city residents corporation officials waste area civic work building municipal roads people garbage

Topic #3:
review bench justice said judge cji temple sc judges court ram ayodhya case apple hc

Topic #4:
searches state said power government strike protest liquor employees workers cows cow solar district electricity

Topic #5:
singh punjab said pakistan jail sad kumar sikh dera village border sahib ludhiana kaur mla

Topic #6:
said mlas congress government meeting bjp karnataka candidates january jd party minister channels exam kumaraswamy

Topic #7:
film says like people time life actor films year work just don love want lot

Topic #8:
said railway airport train passengers station flight trains hours officials delhi time express off

In [11]:
import pandas as pd
doc_topic_dist = pd.DataFrame(lda.transform(count_data))
#doc_topic_dist.to_csv('doc_topic_dist.csv', index=False)

maxValueInd = doc_topic_dist.idxmax(axis = 1)


In [12]:
maxValueInd

0        16
1        20
2         2
3         1
4        21
         ..
75110    10
75111     0
75112     9
75113    28
75114    19
Length: 75115, dtype: int64

In [13]:
maxValueInd.to_csv('j_topics.csv')
df = pd.read_csv('j_topics.csv')
df = df.drop(columns=['Unnamed: 0'])
df

,0
0,16
1,20
2,2
3,1
4,21
...,...
75110,10
75111,0
75112,9
75113,28


In [14]:
df1=(df['0'].value_counts()/df['0'].count())*100
df1.to_csv('freq_j.csv')
df = pd.read_csv('freq_j.csv')
df

,Unnamed: 0,0
0,9,10.441323
1,20,9.042135
2,0,8.486987
3,21,7.102443
4,7,6.360913
5,13,5.082873
6,10,4.370632
7,4,4.004526
8,16,3.874060
9,2,3.846103
